In [1]:
from sklearn.datasets import fetch_openml
#from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score,precision_score,recall_score
import seaborn as sns
#from fairlearn.metrics import selection_rate
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from aif360.datasets import BinaryLabelDataset
from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing import DisparateImpactRemover
from sklearn.metrics import accuracy_score, confusion_matrix
from aif360.metrics import ClassificationMetric
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dense, Dropout, Input
from tqdm import tqdm


!pip install -q BlackBoxAuditing 
!pip install -q aif360

import numpy as np
import pandas as pd
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
import keras
from keras.layers import *
from keras.models import *
from keras.optimizers import Adam
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric

import warnings
warnings.simplefilter(action='ignore', category='PerformanceWarning')


pip install 'aif360[LawSchoolGPA]'
2024-02-11 12:31:36.634833: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
df = pd.read_csv('AV3 2.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

# Preparing data

# Filling missing values

In [4]:
df.Gender.fillna(df.Gender.mode().values[0], inplace = True)

df.Married.fillna(df.Married.mode().values[0], inplace=True)

df.Dependents.fillna(df.Dependents.mode().values[0], inplace=True)

df.Self_Employed.fillna(df.Self_Employed.mode().values[0], inplace=True)

df.LoanAmount.fillna(round(df.LoanAmount.mean(),0),inplace=True)

df.Loan_Amount_Term.fillna(round(df.Loan_Amount_Term.mean(),0), inplace=True)

df.Credit_History.fillna(df.Credit_History.mode().values[0], inplace=True)


In [5]:
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# Dropping ID AND Replacing 3+ enteries

In [6]:
### dropping ID

# Drop Loan_ID
df = df.drop(['Loan_ID'], axis = 1)

### Replacing 3+

### Replacing 3+ in 'Dependents' with 3
df = df.replace({'Dependents': '3+'}, 3)

# Converting Categorical Variables into Numbers  

In [7]:

label_encoder=LabelEncoder()

df['Gender']=label_encoder.fit_transform(df['Gender'])
df['Married']=label_encoder.fit_transform(df['Married'])
df['Education']=label_encoder.fit_transform(df['Education'])
df['Self_Employed']=label_encoder.fit_transform(df['Self_Employed'])
df['Property_Area']=label_encoder.fit_transform(df['Property_Area'])
df['Loan_Status']=label_encoder.fit_transform(df['Loan_Status'])
df['Gender']=label_encoder.fit_transform(df['Gender'])
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,0,0,5849,0.0,146.0,360.0,1.0,2,1
1,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1


# Addressing and Mitigating Bias with AI Fairness 360

# Identifying Bias in the model 

# Splitting data

In [8]:
X=df.drop('Loan_Status',axis=1)
y=df['Loan_Status']
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

# Original Model 

In [9]:
model1 = RandomForestClassifier(n_estimators=100, random_state=42)
model1.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [10]:
y_pred_test = model1.predict(X_test)
y_pred = model1.predict(X)

# Metrics

In [11]:

# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Loan_Status'])


for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([_X_test, _y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = y_pred_test
#np.argmax(y_pred_test).values,axis=1
        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())
            max_di = max(max_di, metric.disparate_impact())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
print("Max Disparate Impact (Ideal value = 1):", max_di)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.1242124212421242
Max Average Odds Difference (Ideal value = 0): 0.0897782772782773
Max Equal Opportunity Difference (Ideal value = 0): 0.18055555555555558
Max Theil Index (Ideal value = 0): 0.19007812277935573
Max Generalized Entropy Index (Ideal value = 0): 0.12393474555636722
Max Disparate Impact (Ideal value = 1): 1.8363636363636364


# Mitigation Algorithms

## Preprocessing Algorithms:

### 1- Reweighing

In [12]:

    
for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset_train = StandardDataset(df=pd.concat([X_train, y_train], axis=1),
                                           label_name='Loan_Status', favorable_classes=[age_cls],
                                           protected_attribute_names=['Gender'],
                                           privileged_classes=[[priv_cls]])
    

    # Define the groups
    groups = [
        {'Gender': 0},
        {'Gender': 1}
        ]

        # Iterate through the groups
    for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

                                          
weights = aif360_dataset_train.instance_weights
preprocessor = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=[privileged_group])
train_dataset_reweighted = preprocessor.fit_transform(aif360_dataset_train)
#metric_e = BinaryLabelDatasetMetric(train_dataset_reweighted, unprivileged_groups=[{sensitive_attribute: 0}], privileged_groups=[{sensitive_attribute: 1}])

model_weigh = RandomForestClassifier(n_estimators=100, random_state=42)
model_weigh.fit(X_train,y_train , sample_weight=weights)
#aif360_dataset_train.features  aif360_dataset_train.labels.ravel()  

RandomForestClassifier(random_state=42)

In [13]:
y_pred_wei = model_weigh.predict(X)

In [14]:
# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
#_X_test = pd.DataFrame(X_test, columns=X.columns)
#_y_test = pd.DataFrame(y_test, columns=['Loan_Status'])


for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X, y], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset1 = aif360_dataset.copy()
        aif360_pred_dataset1.labels = y_pred_wei
#np.argmax(y_pred_wei).values,axis=1
        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]

#aif360_pred_dataset.labels = 

        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset1,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())
            max_di = max(max_di, metric.disparate_impact())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
print("Max Disparate Impact (Ideal value = 1):", max_di)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.012734775184974456
Max Average Odds Difference (Ideal value = 0): 0.007645823754404216
Max Equal Opportunity Difference (Ideal value = 0): 0.011333914559720992
Max Theil Index (Ideal value = 0): 0.03585718471180844
Max Generalized Entropy Index (Ideal value = 0): 0.021826469906513025
Max Disparate Impact (Ideal value = 1): 1.0437866927592954


## 2- Disparate Impact Remover

In [15]:
  
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Loan_Status'])
 

transformed_X_Train = pd.DataFrame(index=X_train.index, columns=X_train.columns)

groups = [
    {'Gender': 0},
    {'Gender': 1},
                  ]
priv_group = groups[0]
unprivileged_groups = [groups[1]]
aif360_dataset = StandardDataset(df,
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

   
DIR = DisparateImpactRemover(repair_level=1.0, sensitive_attribute="Gender")
aif360_dataset_train = DIR.fit_transform(aif360_dataset)         
transformed_X_Train = transformed_X_Train.add(pd.DataFrame(aif360_dataset_train.features, columns=X.columns), fill_value=0)
            


model_DIR = RandomForestClassifier(n_estimators=100, random_state=42)
model_DIR.fit(transformed_X_Train.astype('float32'), y.astype('float32'))

#aif360_dataset_train.features  aif360_dataset_train.labels.ravel() 

RandomForestClassifier(random_state=42)

In [16]:
  
y_pred_DIR= model_DIR.predict(X_test)

In [18]:
# Initialize variables to store maximum values of each metric

max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Loan_Status'])


for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([_X_test, _y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset2 = aif360_dataset.copy()
        aif360_pred_dataset2.labels = y_pred_DIR
#np.argmax(y_pred_wei).values,axis=1
        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]

#aif360_pred_dataset.labels = 

        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset2,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())
            max_di = max(max_di, metric.disparate_impact())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
print("Max Disparate Impact (Ideal value = 1):", max_di)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.2011701170117012
Max Average Odds Difference (Ideal value = 0): 0.10551948051948051
Max Equal Opportunity Difference (Ideal value = 0): 0.25
Max Theil Index (Ideal value = 0): 0.0593499716183689
Max Generalized Entropy Index (Ideal value = 0): 0.03812500000000017
Max Disparate Impact (Ideal value = 1): 1.9675324675324677


# In-processing Algorithms:

## 1- Adversarial Debiasing

In [21]:
 y_pred_8 = y_pred_DIR.copy()

In [26]:

#pip install --upgrade tensorflow
import tensorflow as tf
print(tf.__version__)


2.15.0


In [ ]:
#pip install tensorflow

In [30]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.datasets import StandardDataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tqdm import tqdm


groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]
sess = tf.Session()
            # Convert to AIF360 format
#aif360_dataset = StandardDataset(df=pd.concat([_X_test, _y_test], axis=1),
                                        # label_name='Loan_Status', favorable_classes=[age_cls],
                                        # protected_attribute_names=['Gender'],
                                        # privileged_classes=[[priv_cls]])

# CV and OOF Predictions
#y_pred_4 = pd.DataFrame(0, columns=[0, 1], index=X.index)
scores = []

# Unique values of the target
unique_targets = y.unique()

for train_index, test_index in tqdm(StratifiedKFold(n_splits=5).split(X, y)):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    for target_value in unique_targets:
        tf.reset_default_graph()
        sess = tf.Session()

        # Convert the target to binary for the current class
        y_train_binary = (y_train == target_value).astype(int)
        y_test_binary = (y_test == target_value).astype(int)

        # Convert to AIF360 format
        aif360_train = StandardDataset(pd.concat([X_train, y_train_binary], axis=1),
                                       label_name='Loan_Status', favorable_classes=[1],
                                       protected_attribute_names=['Gender'],
                                       privileged_classes=[[priv_group['Gender']]])

        aif360_test = StandardDataset(pd.concat([X_test, y_test_binary], axis=1),
                                      label_name='Loan_Status', favorable_classes=[1],
                                      protected_attribute_names=['Gender'],
                                      privileged_classes=[[priv_group['Gender']]])

        # Initialize and train Adversarial Debiasing model
        adv_model = AdversarialDebiasing(privileged_groups=[priv_group],
                                         unprivileged_groups=unprivileged_groups,
                                         scope_name='adv_debiasing',adversary_loss_weight=0.01,
                                         sess=sess,
                                         num_epochs=10, 
                                         batch_size=32)
        

        
        aif360_dataset_train1 = adv_model.fit(aif360_train)
        #adv_model.fit(aif360_test)         
        #transformed_X_Train = transformed_X_Train.add(pd.DataFrame(aif360_dataset_train.features, columns=X.columns), fill_value=0)
        #transformed_X_Train = transformed_X_Train.add(pd.DataFrame(aif360_dataset_train1.features, columns=X.columns), fill_value=0) 


        model_DIR1 = RandomForestClassifier(n_estimators=100, random_state=42)
        model_DIR1.fit(X,y)
#aif360_dataset_train.astype('float32'), y.astype('float32')
    #y_pred_4 = adv_model.predict(X).labels
        
        
#sess.close()            
            
     #Make predictions and evaluate
     #   preds = adv_model.predict(X_test).labels
    #    y_pred_4.loc[test_index, target_value] = preds[:, 0]
    #    scores.append(accuracy_score(X.values, preds[:, 0]))

         #Close TensorFlow session
        sess.close()

#accuracy_score(y,np.argmax(y_pred_4.values,axis=1))      


0it [00:00, ?it/s]

epoch 0; iter: 0; batch classifier loss: 187.919739; batch adversarial loss: 1.156157
epoch 1; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 2; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 3; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 4; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 5; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 6; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 7; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 8; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 9; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 0; iter: 0; batch classifier loss: 146.463806; batch adversarial loss: 0.676447
epoch 1; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 2; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 3; iter:

1it [00:02,  2.28s/it]

epoch 0; iter: 0; batch classifier loss: 252.435135; batch adversarial loss: 0.693925
epoch 1; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 2; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 3; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 4; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 5; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 6; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 7; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 8; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 9; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 0; iter: 0; batch classifier loss: 141.625519; batch adversarial loss: 0.702676
epoch 1; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 2; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 3; iter:

2it [00:04,  2.20s/it]

...

3it [00:06,  2.18s/it]

epoch 0; iter: 0; batch classifier loss: 138.670258; batch adversarial loss: 0.914715
epoch 1; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 2; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 3; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 4; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 5; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 6; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 7; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 8; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 9; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 0; iter: 0; batch classifier loss: 136.964539; batch adversarial loss: 0.597468
epoch 1; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 2; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 3; iter:

4it [00:08,  2.22s/it]

epoch 0; iter: 0; batch classifier loss: 763.705322; batch adversarial loss: 0.646738
epoch 1; iter: 0; batch classifier loss: 87.270958; batch adversarial loss: 0.682940
epoch 2; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 3; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 4; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 5; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 6; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 7; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 8; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 9; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 0; iter: 0; batch classifier loss: 125.452286; batch adversarial loss: 0.666347
epoch 1; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epoch 2; iter: 0; batch classifier loss: nan; batch adversarial loss: nan
epo

5it [00:11,  2.21s/it]


In [28]:
# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Loan_Status'])


for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([_X_test, _y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset2 = aif360_dataset.copy()
        aif360_pred_dataset2.labels =y_pred_DIR
        #
#np.argmax(y_pred_4.values,axis=1)


        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]

#aif360_pred_dataset.labels = 

        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset2,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())
            #max_di = max(max_di, metric.disparate_impact())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
#print("Max Disparate Impact (Ideal value = 1):", max_di)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.05042016806722688
Max Average Odds Difference (Ideal value = 0): 0.017378464334393467
Max Equal Opportunity Difference (Ideal value = 0): 0.11770623742454722
Max Theil Index (Ideal value = 0): 0.33245636565819003
Max Generalized Entropy Index (Ideal value = 0): 0.23939508506616264


# 2 - PrejudiceRemover

In [22]:
  
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Loan_Status'])
 
from aif360.algorithms.inprocessing import PrejudiceRemover
transformed_X_Train = pd.DataFrame(index=X.index, columns=X_train.columns)

groups = [
    {'Gender': 0},
    {'Gender': 1},
                  ]
priv_group = groups[0]
unprivileged_groups = [groups[1]]
aif360_dataset = StandardDataset(df,
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

remover = PrejudiceRemover()
train_data_debiased = remover.fit(aif360_dataset)



Traceback (most recent call last):
  File "/Users/macbook/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/kamfadm-2012ecmlpkdd/train_pr.py", line 319, in <module>
    main(opt)
  File "/Users/macbook/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/kamfadm-2012ecmlpkdd/train_pr.py", line 166, in main
    clr = train(X, y, ns, opt)
          ^^^^^^^^^^^^^^^^^^^^
  File "/Users/macbook/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/kamfadm-2012ecmlpkdd/train_pr.py", line 128, in train
    clr.fit(X, y, ns, itype=opt.itype)
  File "/Users/macbook/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/kamfadm-2012ecmlpkdd/fadm/lr/pr.py", line 279, in fit
    self.c_s_ = np.array([np.sum(s == si).astype(np.float)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/macbook/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/kamfadm-2012ecmlpkdd/fadm/lr/pr.py", line 279, in

In [24]:
# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Loan_Status'])


for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([_X_test, _y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset2 = aif360_dataset.copy()
        aif360_pred_dataset2.labels = y_pred_8  
        #
#np.argmax(y_pred_4.values,axis=1)


        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]

#aif360_pred_dataset.labels = 

        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset2,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())
            #max_di = max(max_di, metric.disparate_impact())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
#print("Max Disparate Impact (Ideal value = 1):", max_di)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.2011701170117012
Max Average Odds Difference (Ideal value = 0): 0.10551948051948051
Max Equal Opportunity Difference (Ideal value = 0): 0.25
Max Theil Index (Ideal value = 0): 0.0593499716183689
Max Generalized Entropy Index (Ideal value = 0): 0.03812500000000017


# Postprocessing Algorithms:

## 1- Equalized Odds Postprocessing

In [36]:
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from aif360.algorithms.postprocessing import EqOddsPostprocessing,CalibratedEqOddsPostprocessing
from aif360.datasets import StandardDataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow.keras.backend as K



privileged_groups = [{'Gender': 1}]
unprivileged_groups = [{'Gender': 0}]

n_classes = 4  # Number of classes
scores = []
y_pred_all_classes = np.zeros((len(y), n_classes))

for class_index in range(n_classes):
    for train_index, test_index in tqdm(StratifiedKFold(n_splits=5).split(X, y)):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
   

        # Convert the target to binary for the current class
        y_train_binary = (y_train == target_value).astype(int)
        y_test_binary = (y_test == target_value).astype(int)

        
  

        # Predict using the standard model
        #standard_preds = dense_model.predict(X_Test).flatten()  # Flatten for binary output

        # Convert to AIF360 format for postprocessing
        aif360_test = StandardDataset(pd.concat([X_test[['Gender']], pd.DataFrame(y_test_binary, columns=['Loan_Status'], index=X_test.index)], axis=1),
                                      label_name='Loan_Status', favorable_classes=[1],
                                      protected_attribute_names=['Gender'],
                                      privileged_classes=[[1]])
        aif360_preds = StandardDataset(pd.concat([X_test[['Gender']],pd.DataFrame(y_test, columns=['Loan_Status'], index=X_test.index)], axis=1),
                                       label_name='Loan_Status', favorable_classes=[1],
                                       protected_attribute_names=['Gender'],
                                       privileged_classes=[[1]])

        # Apply Equalized Odds Postprocessing
        eq_odds = CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=42)
        eq_odds = eq_odds.fit(aif360_test, aif360_preds)
        transformed_preds = eq_odds.predict(aif360_preds).labels

        # Store the predictions for the current class
        y_pred_all_classes[test_index, class_index] = transformed_preds.squeeze()

# Convert the predictions for each class into final multi-class predictions
y_pred_final = np.argmax(y_pred_all_classes, axis=1)
overall_accuracy = accuracy_score(y, y_pred_final)

print("Overall Accuracy:", overall_accuracy)


5it [00:00, 40.53it/s]
5it [00:00, 55.12it/s]
5it [00:00, 57.87it/s]
5it [00:00, 57.31it/s]

Overall Accuracy: 0.3127035830618892


In [37]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X, y], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = y_pred_final

        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.0
Max Average Odds Difference (Ideal value = 0): 0.0
Max Equal Opportunity Difference (Ideal value = 0): 0.0
Max Theil Index (Ideal value = 0): 1.162499556119361
Max Generalized Entropy Index (Ideal value = 0): 1.0989583333333337


## 2- Reject Option Classification

In [ ]:
#from aif360.algorithms.postprocessing import RejectOptionClassification


#privileged_groups = [{'Gender': 1}]
#unprivileged_groups = [{'Gender': 0}]

#n_classes = 1  # Number of classes
#scores = []
#y_pred_all_classes = np.zeros((len(y), n_classes))

#for class_index in [0,1]:
 #   for train_index, test_index in tqdm(StratifiedKFold(n_splits=5).split(X, y)):
  #      X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
   #     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
   

        # Convert the target to binary for the current class
    #    y_train_binary = (y_train == target_value).astype(int)
     #   y_test_binary = (y_test == target_value).astype(int)


        ## Initialize and train the model
       # #dense_model = create_model()
       ## dense_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])  # Use binary_crossentropy
       ## dense_model.fit(X_Train, y_Train_binary, epochs=25, batch_size=64, verbose=0)

        ## Predict using the standard model
        ##standard_preds = dense_model.predict(X_Test).flatten()  # Flatten for binary output

        # Convert to AIF360 format for postprocessing
      #  aif360_test = StandardDataset(pd.concat([X_test[['Gender']], pd.DataFrame(y_test_binary, columns=['Loan_Status'],index=X_test.index)], axis=1),
       #                               label_name='Loan_Status', favorable_classes=[1],
        #                              protected_attribute_names=['Gender'],
         #                             privileged_classes=[[1]])
        #aif360_preds = StandardDataset(pd.concat([X_test[['Gender']],pd.DataFrame(y_test_binary, columns=['Loan_Status'], index=X_test.index)], axis=1),
         #                              label_name='Loan_Status', favorable_classes=[1],
          #                             protected_attribute_names=['Gender'],
           #                            privileged_classes=[[1]])

        # Apply Reject Option Classification
        #roc = RejectOptionClassification(unprivileged_groups=[{'Gender': 0}],
         #                                privileged_groups=[{'Gender': 1}],
          #                               low_class_thresh=0.01, high_class_thresh=0.99,
           #                              num_class_thresh=100, num_ROC_margin=50,
            #                             metric_name="Statistical parity difference",
             #                            metric_ub=0.05, metric_lb=-0.05)
        
        
        
        

        #roc = roc.fit(aif360_test, aif360_preds)
        #transformed_preds = roc.predict(aif360_test).labels


        # Store the predictions for the current class
       # y_pred_all_classes[test_index, class_index] = transformed_preds#.squeeze()

# Convert the predictions for each class into final multi-class predictions
#y_pred_final1 = np.argmax(y_pred_all_classes, axis=1)
#overall_accuracy = accuracy_score(y, y_pred_final1)

#print("Overall Accuracy:", overall_accuracy)


# 3- Calibrated EqOdds Postprocessing

In [39]:
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from aif360.algorithms.postprocessing import EqOddsPostprocessing,CalibratedEqOddsPostprocessing
from aif360.datasets import StandardDataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow.keras.backend as K

###############################
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Loan_Status'])
 

transformed_X_Train = pd.DataFrame(index=X_train.index, columns=X_train.columns)

groups = [
    {'Gender': 0},
    {'Gender': 1},
                  ]
priv_group = groups[0]
unprivileged_groups = [groups[1]]
aif360_dataset = StandardDataset(df,
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

   
DIR = DisparateImpactRemover(repair_level=1.0, sensitive_attribute="Gender")
aif360_dataset_train = DIR.fit_transform(aif360_dataset)         
transformed_X_Train = transformed_X_Train.add(pd.DataFrame(aif360_dataset_train.features, columns=X.columns), fill_value=0)
            


model_DIR = RandomForestClassifier(n_estimators=100, random_state=42)
model_DIR.fit(transformed_X_Train.astype('float32'), y.astype('float32'))

#aif360_dataset_train.features  aif360_dataset_train.labels.ravel() 



#################################

privileged_groups = [{'Gender': 1}]
unprivileged_groups = [{'Gender': 0}]

n_classes = 4  # Number of classes
scores = []
y_pred_all_classes = np.zeros((len(y), n_classes))

for class_index in range(n_classes):
    for train_index, test_index in tqdm(StratifiedKFold(n_splits=5).split(X, y)):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
   

        # Convert the target to binary for the current class
        y_train_binary = (y_train == target_value).astype(int)
        y_test_binary = (y_test == target_value).astype(int)

        
  

        # Predict using the standard model
        #standard_preds = dense_model.predict(X_Test).flatten()  # Flatten for binary output

        # Convert to AIF360 format for postprocessing
        aif360_test = StandardDataset(pd.concat([X_test[['Gender']], pd.DataFrame(y_test_binary, columns=['Loan_Status'], index=X_test.index)], axis=1),
                                      label_name='Loan_Status', favorable_classes=[1],
                                      protected_attribute_names=['Gender'],
                                      privileged_classes=[[1]])
        aif360_preds = StandardDataset(pd.concat([X_test[['Gender']],pd.DataFrame(y_test, columns=['Loan_Status'], index=X_test.index)], axis=1),
                                       label_name='Loan_Status', favorable_classes=[1],
                                       protected_attribute_names=['Gender'],
                                       privileged_classes=[[1]])
        
        
        calibrated_eq_odds = CalibratedEqOddsPostprocessing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}])
calibrated_eq_odds.fit(aif360_test, aif360_preds)

# Obtain the updated predictions
predictions_calibrated_eq_odds = calibrated_eq_odds.predict(aif360_test, threshold=0.5)
        #CEOP = CEOP.fit(aif360_test, aif360_preds)
        #transformed_preds = CEOP.predict(aif360_test).labels
        
        #CEOP = CEOP.fit(aif360_test, aif360_preds)
        #transformed_preds = CEOP.predict(aif360_test, threshold=0.5).labels

        # Store the predictions for the current class
        #y_pred_all_classes[test_index, class_index] = transformed_preds.squeeze()

# Convert the predictions for each class into final multi-class predictions
#y_pred_final1 = np.argmax(y_pred_all_classes, axis=1)
#overall_accuracy = accuracy_score(y, y_pred_final1)
#y_pred_all_classes[test_index, class_index] = predictions_calibrated_eq_odds.squeeze()

# Convert the predictions for each class into final multi-class predictions
#y_pred_final5 = predictions_calibrated_eq_odds.fit(X_test)
#overall_accuracy = accuracy_score(y, y_pred_final5)
#print("Overall Accuracy:", overall_accuracy)


5it [00:00, 73.51it/s]
5it [00:00, 62.15it/s]
5it [00:00, 65.58it/s]
5it [00:00, 64.89it/s]


In [ ]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([_X_test, _y_test], axis=1),
                                         label_name='Loan_Status', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = y_pred  
        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)